In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from pprint import pprint

from tiled.client import from_uri
from tiled.queries import Contains

import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
# mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.usetex'] = False
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

In [ ]:
from lightway.ingest.iss import ingest_all
from lightway.postprocessing.operations import StandardizeGrid, NormalizeLarch, QAQC, postprocess

With a tiled server spun up, get the client.

In [ ]:
client = from_uri("http://127.0.0.1:8000?api_key=eli1is2the3big4cheese")

Load everything into the client. This performs local validation to make sure everything contains the correct metadata, columns, etc. It will throw errors and kill the program if things do not satisfy all validation steps.

In [ ]:
ingest_all(client, "real_example_data")

Get all of the e.g. transmission data for Ni, and setup two operators for postprocessing procedures.
1. Standardize the grid appropriately
2. Normalize using larch

In [ ]:
transmission_node_Ni = client.search(Contains("channel", "transmission")).search(Contains("Element-symbol", "Ni"))

There is a lot of crap in this database.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for key, value in transmission_node_Ni.items():
    d = value.read()
    ax.plot(d["energy"], d["mu"])
    
plt.show()

Quality assurance and control.

In [ ]:
postprocess(transmission_node_Ni, operators=[NormalizeLarch(), StandardizeGrid(x0=8240.0, xf=8800.0, nx=300), QAQC()])

In [ ]:
transmission_node_Ni_good = transmission_node_Ni.search(Contains("quality", "good"))

Much better. Still some weirdness here but this is what we want to see.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for key, value in transmission_node_Ni_good.items():
    d = value.read()
    ax.plot(d["energy"], d["mu"])
    
plt.show()